In [ ]:
import cv2
import imutils
import glob
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision.models.detection.rpn import AnchorGenerator
!pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import json

from google.colab.patches import cv2_imshow

In [ ]:
# Requirement for SORT algorithm
!pip install filterpy==1.4.5
!pip install scikit-image==0.17.2
!pip install lap==0.4.0

# Faster R-CNN models

In [ ]:
def get_MobileNetmodel(trained=True, save_path=None):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
  # print(model)

  num_classes = 2  # 1 class (bird) + background

  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features

  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  if trained:
    if save_path == None: print("No path to the saved model")
    else: model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))
  model.to(device)
  return model

model = get_MobileNetmodel(trained=False)
# print(model)

def get_MobileNet320model(trained=True, save_path=None):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)

  num_classes = 2  # 1 class (bird) + background

  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features

  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
  model.to(device)
  if trained:
    if save_path == None: print("No path to the saved model")
    else: model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))
  
  return model

def get_ResNet50model(trained=True, save_path=None):
  #device = 'cpu'
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  print("Device", device)
  
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  #model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=True)
  num_classes = 2  # 1 class (bird) + background

  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features

  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  if trained:
      if save_path == None: print("No path to the saved model")
      else: model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))

  model.to(device)
  return model

def transforminput(maxsize):
    transforms = []
    # transforms.append(A.PadIfNeeded(maxsize, maxsize, border_mode=cv2.BORDER_CONSTANT))
    # transforms.append(A.Resize(576, 576))
    transforms.append(A.CenterCrop(width=576, height=576))
    return A.Compose(transforms)

def transformback(size):
    transforms = []
    transforms.append(A.Resize(size[0], size[1]))        
    transforms.append(ToTensorV2(p=1.0)) 
    return A.Compose(transforms, bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

toTensor = T.Compose([T.ToTensor()])

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth


  0%|          | 0.00/74.2M [00:00<?, ?B/s]

# SORT algotihm (test - not clean)

In [ ]:
def video_detection(videopath):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = get_ResNet50model(trained=True, save_path=ROOT_DIR_SAVING + MODEL_NAME)
  model.eval()
  vid = cv2.VideoCapture(videopath)
  all_frames = []
  all_boxes = []
  all_scores = [] 
  while vid.isOpened():
      ret, orig_frame = vid.read()
      if ret == True:
          orig_shape = orig_frame.shape
          frame = cv2.cvtColor(orig_frame, cv2.COLOR_BGR2RGB).astype(np.float32)
          frame /= 255.0
          frame = cv2.resize(frame, (576,576))
          tensorframe = toTensor(frame)
          tensorframe = tensorframe[None, ...]
          outputs = model(tensorframe)
          outputs = [{k: v.to(device) for k, v in t.items()} for t in outputs]


          score  = outputs[0]['scores']
          predboxes = outputs[0]['boxes']
          labels = outputs[0]["labels"].cpu().detach().numpy().astype(np.int32)
          print("Score", score)
          # print("True boxes", boxes)

          keepidx = torchvision.ops.nms(predboxes, score, 0.1).cpu().detach().numpy().astype(np.int32)
          score  = outputs[0]['scores'].cpu().detach().numpy()
          output = outputs[0]['boxes'].cpu().detach().numpy().astype(np.int32)
          score = score[keepidx]
          output = output[keepidx]
          labels = labels[keepidx]

          confidenceidx = [idx for idx, elt in enumerate(score) if elt >= 0.5]
          output = output[confidenceidx]
          score = score[confidenceidx]
          labels = labels[confidenceidx]
          print("NMSScore", score)


          area = (output[:, 3] - output[:, 1]) * (output[:, 2] - output[:, 0])
          zeroarea_filter = []
          for i in area:
              if i == 0:
                  zeroarea_filter.append(False)
              else:
                  zeroarea_filter.append(True)

          sample = {
              'image': frame,
              'bboxes': output[zeroarea_filter],
              'labels': labels[zeroarea_filter]
          }

          size = (orig_shape[0], orig_shape[1]) # height, width
          print("Size of original image", size)
          sample = transformback(size)(**sample)
          boxes = sample['bboxes']

          all_frames.append(orig_frame)
          all_boxes.append(boxes)
          all_scores.append(score)
      else:
          print("Error - Problem with reading the video")
          break
  return all_frames, all_boxes, all_scores

ROOT_DIR_SAVING = "/content/drive/MyDrive/Thesis/savedmodel/"
MODEL_NAME = "fasterrcnn_SGD0005_SchedulerNone_Size576_Batch8_Epoch50_8020.pth"
videopath = "/content/drive/MyDrive/Thesis/videos/A6.mp4"
all_frames, all_boxes, all_scores = video_detection(videopath)

In [ ]:
def getSORTformat(all_boxes, all_scores):
  all_SORTboxes = []
  for idx_frame, i in enumerate(all_boxes): # i is list boxes in frame
    frame = []
    for idx_obj, obj in enumerate(i): # obj is box in list boxes
      obj = list(obj)
      frame.append(list(obj) + [all_scores[idx_frame][idx_obj]])
    all_SORTboxes.append(frame)
  return all_SORTboxes

all_frames, all_boxes, all_scores = all_frames.copy(), all_boxes.copy(), all_scores.copy()
all_SORTboxes = getSORTformat(all_boxes, all_scores)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/')
from sort import *

#create instance of SORT
mot_tracker = Sort(max_age=10, min_hits=1) 

# get detections
#...

# update SORT
all_updateboxes = []
initial_frame = True
for i in all_SORTboxes:
  if (len(i) == 0) & initial_frame:
    track_bbs_ids = mot_tracker.update(np.array(i))
  if (len(i) != 0) & initial_frame:
    initial_frame = False
  if (len(i) == 0) & (not initial_frame):
    i = np.array([[0,0,0,0,0]])
    track_bbs_ids = mot_tracker.update(np.array(i))
  else:
    track_bbs_ids = mot_tracker.update(np.array(i))
  all_updateboxes.append(track_bbs_ids)
# print(all_updateboxes[0:15])

/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/sort.py:74: RuntimeWarning: invalid value encountered in true_divide
  r = w / float(h)


In [ ]:
all_box_frames = []
for idx_frame, orig_frame in enumerate(all_frames):
  # print(idx_frame)
  for idx, box in enumerate(all_updateboxes[idx_frame]):
    print(box)
    cv2.rectangle(orig_frame,
                (int(box[0]), int(box[1])),
                (int(box[2]), int(box[3])),
                (255, 0, 0), 5)
  # print("ok")
  imS = cv2.resize(orig_frame, (960, 540))
  all_box_frames.append(orig_frame)
  cv2_imshow(imS)

In [ ]:
SAVE_PATH = "/content/drive/MyDrive/Thesis/videos/prediction/predictiontrackingA5_Batch8_Epoch50_576.mp4"
size = (4096, 2160)
out = cv2.VideoWriter(SAVE_PATH,cv2.VideoWriter_fourcc(*'mp4v'), 20, size)
 
for i in range(len(all_box_frames)):
    out.write(all_box_frames[i])
out.release()

# Video Detection

In [ ]:
###################
# Video Detection #
###################
# ROOT_DIR_SAVING = "/content/drive/MyDrive/Thesis/savedmodel/"
# MODEL_NAME = "bestfasterrcnnv2_SGD0005_SchedulerReduceLR_Size576_Batch8_Epoch50_8020_rc.pth"
# MODEL_NAME = "bestMobileNetv3_SGD00005_SchedulerReduceLR_Size576_Batch8_Epoch100_8020_rc.pth"
# MODEL_NAME = "bestMobileNetv3320_SGD00005_SchedulerReduceLR_Size576_Batch8_Epoch100_8020_rc.pth"

ROOT_DIR_SAVING = "/content/drive/MyDrive/Thesis/savedmodel/v1/"
MODEL_NAME = "bestfasterrcnnv1_SGD0005_SchedulerReduceLR_Size576_Batch8_Epoch100.pth"
# MODEL_NAME = "bestfasterrcnnv1_MobileNetv3_SGD00005_SchedulerReduceLR_Size576_Batch8_Epoch100.pth"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# model = get_ResNet50model(trained=True, save_path=ROOT_DIR_SAVING + MODEL_NAME)
model = get_MobileNetmodel(trained=True, save_path=ROOT_DIR_SAVING + MODEL_NAME)
model.eval()

print("------------Load------------")

# vid = cv2.VideoCapture("../dataset/Personal_videos/videos/VID_20211105_120528.mp4")
# vid = cv2.VideoCapture("/content/drive/MyDrive/Thesis/videos/test1.mp4")
# vid = cv2.VideoCapture("/content/drive/MyDrive/Thesis/videos/test2.mp4")
vid = cv2.VideoCapture("/content/drive/MyDrive/Thesis/videos/vid7.mp4")
# cv2.namedWindow("img", cv2.WINDOW_NORMAL)
iter = 0
all_frames = []
all_boxes = []
all_birdamt = []
imgcount = 1
while vid.isOpened():
    # print("Enter")
    ret, orig_frame = vid.read()
    # print(ret)
    if ret == True:
        orig_shape = orig_frame.shape
        size = (orig_shape[0], orig_shape[1])
        frame = cv2.cvtColor(orig_frame, cv2.COLOR_BGR2RGB).astype(np.float32)
        # frame = transforminput(max(orig_shape))(image=frame)['image']
        # frame = frame[0:0+1576, 0:0+1576]
        # cv2_imshow(frame)
        # cv2.imwrite("/content/drive/MyDrive/Thesis/videos/video2image/v5/v5img"+str(imgcount)+".jpg", orig_frame)
        # imgcount += 1

        frame /= 255.0
        # frame = cv2.resize(frame, (32*192,32*108))
        tensorframe = toTensor(frame)
        tensorframe = tensorframe[None, ...].cuda()
        outputs = model(tensorframe)
        outputs = [{k: v.to(device) for k, v in t.items()} for t in outputs]

        score  = outputs[0]['scores']
        predboxes = outputs[0]['boxes']
        labels = outputs[0]["labels"].cpu().detach().numpy().astype(np.int32)
        print("Score", score)
        # print("True boxes", boxes)

        keepidx = torchvision.ops.nms(predboxes, score, 0.2).cpu().detach().numpy().astype(np.int32)
        score  = outputs[0]['scores'].cpu().detach().numpy()
        output = outputs[0]['boxes'].cpu().detach().numpy().astype(np.int32)
        score = score[keepidx]
        output = output[keepidx]
        labels = labels[keepidx]

        confidenceidx = [idx for idx, elt in enumerate(score) if elt >= 0.1]
        output = output[confidenceidx]
        score = score[confidenceidx]
        labels = labels[confidenceidx]
        print("NMSScore", score)

        sample = {
            'image': frame,
            'bboxes': output,
            'labels': labels
        }
        print(size)
        sample = transformback(size)(**sample)
        boxes = sample['bboxes']

        # loop over the boxes
        for idx, box in enumerate(boxes):
            cv2.rectangle(orig_frame,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        (255, 0, 0), 5)
            all_birdamt.append(len(boxes))
        #     cv2.putText(orig_frame, 'Bird '+ str(round(score[idx],3)), (int(box[0]), int(box[1])-15), cv2.FONT_HERSHEY_SIMPLEX, 1.6, (255,0,0), 3)
        # for box in output:
        #     cv2.rectangle(frame,
        #                 (int(box[0]), int(box[1])),
        #                 (int(box[2]), int(box[3])),
        #                 (0, 0, 255), 1)
        all_frames.append(orig_frame)
        all_boxes.append(boxes)
        # else: break
        # print("Frame" + str(iter))
        iter += 1
        imS = cv2.resize(orig_frame, (960, 540))
        cv2_imshow(imS)
        cv2.waitKey(1)
        # # Press Q on keyboard to  exit
        # if cv2.waitKey(500) & 0xFF == ord('q'):
        #     break
        if 0xFF == ord('q'):
            break
    else:
        break

print("Save the all the boxes and the counting into a file csv")
PSAVE_PATH = "/content/drive/MyDrive/Thesis/videos/presentationvideos/cigonyes.csv"
data = np.asarray([[all_birdamt, all_boxes]])
np.savetxt(PSAVE_PATH, data, delimiter=',')

# Load the txt file
data = np.loadtxt(PSAVE_PATH, delimiter=',')
print(data)


print("----------Prediction Done----------")
# SAVE_PATH = "/content/drive/MyDrive/Thesis/videos/prediction/predictionA10_Batch8_Epoch50_224.mp4"
SAVE_PATH = "/content/drive/MyDrive/Thesis/videos/prediction/predvid7Mobile.mp4"
size = (size[1], size[0])
out = cv2.VideoWriter(SAVE_PATH,cv2.VideoWriter_fourcc(*'mp4v'), 20, size)
 
for i in range(len(all_frames)):
    out.write(all_frames[i])
out.release()
print("--------------Saved----------------")

In [ ]:
# Code to save boxes/bird amount : better than saving the processed video
PSAVE_PATH = "/content/drive/MyDrive/Thesis/videos/presentationvideos/cigonyes.npy"
all_birdamt = [50,80,50,90,60,50]
all_boxes = [[[2,5,1,6]], [[4,5,8,45], [4,9,0,70]]]
data = np.asarray([all_birdamt, all_boxes])
import pickle as cPickle
with open(PSAVE_PATH, 'wb') as outp:  # Overwrites any existing file.
    cPickle.dump(data, outp, cPickle.HIGHEST_PROTOCOL)

# Load the txt file
file = open(PSAVE_PATH,'rb')
dataPickle = file.read()
file.close()
res = cPickle.loads(dataPickle)
print(res)
print(res[0])

[list([50, 80, 50, 90, 60, 50])
 list([[[2, 5, 1, 6]], [[4, 5, 8, 45], [4, 9, 0, 70]]])]
[50, 80, 50, 90, 60, 50]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


# Image Detection

In [ ]:
###################
# Image Detection #
###################
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
ROOT_DIR_SAVING = "/content/drive/MyDrive/Thesis/savedmodel/"
# MODEL_NAME = "bestfasterrcnnv2_SGD0005_SchedulerReduceLR_Size576_Batch8_Epoch50_8020_rc.pth"
MODEL_NAME = "bestMobileNetv3_SGD00005_SchedulerReduceLR_Size576_Batch8_Epoch100_8020_rc.pth"
# MODEL_NAME = "bestMobileNetv3320_SGD00005_SchedulerReduceLR_Size576_Batch8_Epoch100_8020_rc.pth"

# ROOT_DIR_SAVING = "/content/drive/MyDrive/Thesis/savedmodel/v1/"
# MODEL_NAME = "bestfasterrcnnv1_SGD0005_SchedulerReduceLR_Size576_Batch8_Epoch100.pth"
# MODEL_NAME = "bestfasterrcnnv1_MobileNetv3_SGD00005_SchedulerReduceLR_Size576_Batch8_Epoch100.pth"
# MODEL_NAME = "bestfasterrcnnv1_MobileNetv3320_SGD00005_SchedulerReduceLR_Size576_Batch8_Epoch100.pth"
# MODEL_NAME = "fasterrcnn_SGD0005_SchedulerNone_Size224_Batch8_Epoch25.pth"
# MODEL_NAME = "bestfasterrcnnv2_SGD0005_SchedulerReduceLR_Size576_Batch8_Epoch50_8020_rc.pth"
# model = get_ResNet50model(trained=True, save_path=ROOT_DIR_SAVING + MODEL_NAME)
model = get_MobileNetmodel(trained=True, save_path=ROOT_DIR_SAVING + MODEL_NAME)
model.eval()

def imagedetection(orig_frame):
  orig_frame = orig_frame[120:120+576, 600:600+576] #Pigeon
  # orig_frame = orig_frame[1000:1000+576, 1800:1800+576] #v3
  # orig_frame = orig_frame[700:700+576, 1850:1850+576] #v1
  # orig_frame = orig_frame[100:100+576, 530:530+576] #imgAU1
  # cv2_imshow(orig_frame)
  orig_shape = orig_frame.shape
  print(orig_shape)
  frame = cv2.cvtColor(orig_frame, cv2.COLOR_BGR2RGB).astype(np.float32)
  frame /= 255.0
  print(frame.shape)
  # frame = cv2.resize(frame, (576, 576))
  # cv2_imshow(frame*255)
  # cv2_imshow(frame*255)
  tensorframe = toTensor(frame)
  tensorframe = tensorframe[None, ...].to(device)
  
  start = time.time()
  outputs = model(tensorframe)
  end = time.time()
  print("Time for one image", end - start)
  outputs = [{k: v.to(device) for k, v in t.items()} for t in outputs]

  score  = outputs[0]['scores']
  predboxes = outputs[0]['boxes']

  keepidx = torchvision.ops.nms(predboxes, score, 0.2).cpu().detach().numpy().astype(np.int32)
  score  = outputs[0]['scores'].cpu().detach().numpy()
  output = outputs[0]['boxes'].cpu().detach().numpy().astype(np.int32)
  score = score[keepidx]
  output = output[keepidx]

  confidenceidx = [idx for idx, elt in enumerate(score) if elt >= 0.1]
  output = output[confidenceidx]
  score = score[confidenceidx]

  fig, ax = plt.subplots(1, 1, figsize=(16, 8))
  sample = cv2.cvtColor(orig_frame, cv2.COLOR_BGR2RGB)
  print("Size", sample.shape)
  for box in output:
      cv2.rectangle(sample,
                  (int(box[0]), int(box[1])),
                  (int(box[2]), int(box[3])),
                  (0, 0, 255), 2)
      # cv2.putText(sample, str(round(score[idx],3)), (int(box[0]), int(box[1])-6), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (1,0,0), 1)
      
  ax.imshow((sample).astype(np.uint8))
  # plt.savefig("/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/images/"+str(image_id)+"area" + ".png")
  ax.axis('off')
  # plt.savefig("/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/images/imagerapport/imgpigeonv2.pdf", bbox_inches='tight')
  plt.show()

# orig_frame = cv2.imread('/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/dataset/all_images/96.jpg', 1)
# orig_frame = cv2.imread('/content/drive/MyDrive/Thesis/videos/video2image/v5/v5img12.jpg', 1)
# orig_frame = cv2.imread('/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/images/imgAU1.jpg', 1)
orig_frame = cv2.imread('/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/images/ngbird1.JPG', 1)
# orig_frame = cv2.imread('/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/images/flyingbirdimg1.jpg', 1)
# orig_frame = cv2.imread('/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/images/COCO1.jpg', 1)
imagedetection(orig_frame)